In [ ]:
#| default_exp core

# GhApi details

> Detailed information on the GhApi API

In [ ]:
#| export
from fastcore.all import *
from ghapi.metadata import funcs

import mimetypes,base64
from inspect import signature,Parameter,Signature
from urllib.request import Request
from urllib.error import HTTPError
from urllib.parse import quote
from datetime import datetime, timedelta, timezone
from pprint import pprint
from time import sleep
import os, shutil, tempfile, subprocess, fnmatch

In [ ]:
#| hide
from nbdev import *

In [ ]:
#| export
GH_HOST = os.getenv('GH_HOST', "https://api.github.com")
_DOC_URL = 'https://docs.github.com/'

You can set an environment variable named `GH_HOST` to override the default of `https://api.github.com` incase you are running [GitHub Enterprise](https://github.com/enterprise)(GHE).  However, this library has not been tested on GHE, so proceed at your own risk.

In [ ]:
#| export
def _preview_hdr(preview): return {'Accept': f'application/vnd.github.{preview}-preview+json'} if preview else {}

def _mk_param(nm, **kwargs): return Parameter(nm, kind=Parameter.POSITIONAL_OR_KEYWORD, **kwargs)
def _mk_sig_detls(o):
    res = {}
    if o[0]!=object: res['annotation']=o[0]
    res['default'] = o[1] if len(o)>1 else None
    return res
def _mk_sig(req_args, opt_args, anno_args):
    params =  [_mk_param(k) for k in req_args]
    params += [_mk_param(k, default=v) for k,v in opt_args.items()]
    params += [_mk_param(k, **_mk_sig_detls(v)) for k,v in anno_args.items()]
    return Signature(params)

def _decode_response(path: str) -> bool:
    "checks if a endpoint needs to have it's response from `fastcore.core.urlsend` decoded or just return json"
    needs_decode = (
        "/orgs/{org}/migrations/{migration_id}/archive",
        "/repos/{owner}/{repo}/actions/artifacts/{artifact_id}/{archive_format}",
        "/repos/{owner}/{repo}/tarball/{ref}",
        "/repos/{owner}/{repo}/zipball/{ref}",
    )
    return path not in needs_decode

class _GhObj: pass

In [ ]:
#| export
class _GhVerb(_GhObj):
    __slots__ = 'path,verb,tag,name,summary,url,route_ps,params,data,preview,client,decode,__doc__'.split(',')
    def __init__(self, path, verb, oper, summary, url, params, data, preview, client, kwargs):
        tag,*name = oper.split('/')
        name = '__'.join(name)
        name = name.replace('-','_')
        decode = _decode_response(path)
        path,_,_ = partial_format(path, **kwargs)
        route_ps = stringfmt_names(path)
        __doc__ = summary
        data = {o[0]:o[1:] for o in data}
        store_attr()

    def __call__(self, *args, headers=None, **kwargs):
        headers = {**_preview_hdr(self.preview),**(headers or {})}
        d = list(self.data)
        flds = [o for o in self.route_ps+self.params+d if o not in kwargs]
        for a,b in zip(args,flds): kwargs[b]=a
        route_p,query_p,data_p = [{p:kwargs[p] for p in o if p in kwargs}
                                 for o in (self.route_ps,self.params,d)]
        return self.client(self.path, self.verb, headers=headers, decode=self.decode, route=route_p, query=query_p, data=data_p)

    def __str__(self): return f'{self.tag}.{self.name}{signature(self)}\n{self.doc_url}'
    @property
    def __signature__(self): return _mk_sig(self.route_ps, dict.fromkeys(self.params), self.data)
    __call__.__signature__ = __signature__
    @property
    def doc_url(self): return _DOC_URL + self.url.replace(" ","_")

    def _repr_markdown_(self):
        params = ', '.join(self.route_ps+self.params+list(self.data))
        return f'[{self.tag}.{self.name}]({self.doc_url})({params}): *{self.summary}*'
    __repr__ = _repr_markdown_

In [ ]:
#| export
class _GhVerbGroup(_GhObj):
    def __init__(self, name, verbs):
        self.name,self.verbs = name,verbs
        for o in verbs: setattr(self, o.name, o)
    def __str__(self): return "\n".join(str(v) for v in self.verbs)
    def _repr_markdown_(self): return "\n".join(f'- {v._repr_markdown_()}' for v in self.verbs)

In [ ]:
#| export
_docroot = 'https://docs.github.com/rest/reference/'

In [ ]:
#| export
def print_summary(req:Request):
    "Print `Request.summary` with the token (if any) removed"
    pprint(req.summary('Authorization'))

## GhApi -

In [ ]:
#| export
class GhApi(_GhObj):
    def __init__(self, owner=None, repo=None, token=None, jwt_token=None, debug=None, limit_cb=None, gh_host=None,
                 authenticate=True, **kwargs):
        self.headers = { 'Accept': 'application/vnd.github.v3+json' }
        if authenticate:
            token = token or os.getenv('GITHUB_TOKEN', None)
            jwt_token = jwt_token or os.getenv('GITHUB_JWT_TOKEN', None)
            if jwt_token: self.headers['Authorization'] = 'Bearer ' + jwt_token
            elif token: self.headers['Authorization'] = 'token ' + token
            else: warn('Neither GITHUB_TOKEN nor GITHUB_JWT_TOKEN found: running as unauthenticated')
        if owner: kwargs['owner'] = owner
        if repo:  kwargs['repo' ] = repo
        funcs_ = L(funcs).starmap(_GhVerb, client=self, kwargs=kwargs)
        self.func_dict = {f'{o.path}:{o.verb.upper()}':o for o in funcs_}
        self.groups = {k.replace('-','_'):_GhVerbGroup(k,v) for k,v in groupby(funcs_, 'tag').items()}
        self.debug,self.limit_cb,self.limit_rem = debug,limit_cb,5000
        self.gh_host = gh_host or GH_HOST

    def __call__(self, path:str, verb:str=None, headers:dict=None, route:dict=None, query:dict=None, data=None, timeout=None, decode=True):
        "Call a fully specified `path` using HTTP `verb`, passing arguments to `fastcore.core.urlsend`"
        if verb is None: verb = 'POST' if data else 'GET'
        headers = {**self.headers,**(headers or {})}
        if not path.startswith(('http://', 'https://')):
            path = self.gh_host + path
        if route:
            for k,v in route.items(): route[k] = quote(str(route[k]), safe='')
        return_json = ('json' in headers['Accept']) and (decode is True)
        debug = self.debug if self.debug else print_summary if os.getenv('GHAPI_DEBUG') else None
        res,self.recv_hdrs = urlsend(path, verb, headers=headers or None, decode=decode, debug=debug, return_headers=True,
                                     route=route or None, query=query or None, data=data or None, return_json=return_json, timeout=timeout)
        if 'X-RateLimit-Remaining' in self.recv_hdrs:
            newlim = self.recv_hdrs['X-RateLimit-Remaining']
            if self.limit_cb is not None and newlim != self.limit_rem:
                self.limit_cb(int(newlim),int(self.recv_hdrs['X-RateLimit-Limit']))
            self.limit_rem = newlim

        return dict2obj(res) if return_json else res

    def __dir__(self): return super().__dir__() + list(self.groups)
    def _repr_markdown_(self): return "\n".join(f"- [{o}]({_docroot + o.replace('_', '-')})" for o in sorted(self.groups))
    def __getattr__(self,k): return self.groups[k] if 'groups' in vars(self) and k in self.groups else stop(AttributeError(k))

    def __getitem__(self, k):
        "Lookup and call an endpoint by path and verb (which defaults to 'GET')"
        a,b = k if isinstance(k,tuple) else (k,'GET')
        return self.func_dict[f'{a}:{b.upper()}']

    def full_docs(self):
        return '\n'.join(f'## {gn}\n\n{group._repr_markdown_()}\n' for gn,group in sorted(self.groups.items()))

In [ ]:
#| hide
token = os.environ['GITHUB_TOKEN']

### Access by path

In [ ]:
show_doc(GhApi.__call__)

---

[source](https://github.com/fastai/ghapi/blob/main/ghapi/core.py#L123){target="_blank" style="float:right; font-size:smaller"}

### GhApi.__call__

>      GhApi.__call__ (path:str, verb:str=None, headers:dict=None,
>                      route:dict=None, query:dict=None, data=None,
>                      timeout=None, decode=True)

*Call a fully specified `path` using HTTP `verb`, passing arguments to `fastcore.core.urlsend`*

In [ ]:
api = GhApi()

You can call a `GhApi` object as a function, passing in the path to the endpoint, the HTTP verb, and any route, query parameter, or post data parameters as required.

In [ ]:
api('/repos/{owner}/{repo}/git/ref/{ref}', 'GET', route=dict(
    owner='fastai', repo='ghapi-test', ref='heads/master'))

```json
{ 'node_id': 'MDM6UmVmMzE1NzEyNTg4OnJlZnMvaGVhZHMvbWFzdGVy',
  'object': { 'sha': '958659bf10a8c275fb04f2f6832f1a0f4d07fa85',
              'type': 'commit',
              'url': 'https://api.github.com/repos/fastai/ghapi-test/git/commits/958659bf10a8c275fb04f2f6832f1a0f4d07fa85'},
  'ref': 'refs/heads/master',
  'url': 'https://api.github.com/repos/fastai/ghapi-test/git/refs/heads/master'}
```

In [ ]:
show_doc(GhApi.__getitem__)

---

[source](https://github.com/fastai/ghapi/blob/main/ghapi/core.py#L147){target="_blank" style="float:right; font-size:smaller"}

### GhApi.__getitem__

>      GhApi.__getitem__ (k)

*Lookup and call an endpoint by path and verb (which defaults to 'GET')*

You can access endpoints by indexing into the object. When using the API this way, you do not need to specify what type of parameter (route, query, or post data) is being used. This is, therefore, the same call as above:

In [ ]:
api['/repos/{owner}/{repo}/git/ref/{ref}'](owner='fastai', repo='ghapi-test', ref='heads/master')

```json
{ 'node_id': 'MDM6UmVmMzE1NzEyNTg4OnJlZnMvaGVhZHMvbWFzdGVy',
  'object': { 'sha': '958659bf10a8c275fb04f2f6832f1a0f4d07fa85',
              'type': 'commit',
              'url': 'https://api.github.com/repos/fastai/ghapi-test/git/commits/958659bf10a8c275fb04f2f6832f1a0f4d07fa85'},
  'ref': 'refs/heads/master',
  'url': 'https://api.github.com/repos/fastai/ghapi-test/git/refs/heads/master'}
```

### Media types

For some endpoints GitHub lets you specify a [media type](https://docs.github.com/en/rest/overview/media-types) the for response data, using the `Accept` header. If you choose a media type that is not JSON formatted (for instance `application/vnd.github.v3.sha`) then the call to the `GhApi` object will return a string instead of an object.

In [ ]:
api('/repos/{owner}/{repo}/commits/{ref}', 'GET', route=dict(
    owner='fastai', repo='ghapi-test', ref='refs/heads/master'),
    headers={'Accept': 'application/vnd.github.VERSION.sha'})

'958659bf10a8c275fb04f2f6832f1a0f4d07fa85'

### Rate limits

GitHub has various [rate limits](https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting) for their API. After each call, the response includes information about how many requests are remaining in the hourly quota. If you'd like to add alerts, or indications showing current quota usage, you can register a callback with `GhApi` by passing a callable to the `limit_cb` parameter. This callback will be called whenever the amount of quota used changes. It will be called with two arguments: the new quota remaining, and the total hourly quota.

In [ ]:
def _f(rem,quota): print(f"Quota remaining: {rem} of {quota}")

api = GhApi(limit_cb=_f)
api['/repos/{owner}/{repo}/git/ref/{ref}'](owner='fastai', repo='ghapi-test', ref='heads/master').ref

Quota remaining: 4907 of 5000


'refs/heads/master'

You can always get the remaining quota from the `limit_rem` attribute:

In [ ]:
api.limit_rem

'4907'

## Operations

Instead of passing a path to `GhApi`, you will more often use the operation methods provided in the API's operation groups, which include documentation, signatures, and auto-complete.

If you provide `owner` and/or `repo` to the constructor, they will be automatically inserted into any calls which use them (except when calling `GhApi` as a function). You can also pass any other arbitrary keyword arguments you like to have them used as defaults for any relevant calls.

You must include a GitHub API token if you need to access any authenticated endpoints. If don't pass the `token` param, then your `GITHUB_TOKEN` environment variable will be used, if available.

In [ ]:
api = GhApi(owner='fastai', repo='ghapi-test', token=token)

### Operation groups

The following groups of endpoints are provided, which you can list at any time along with a link to documentation for all endpoints in that group, by displaying the `GhApi` object:

In [ ]:
api

- [actions](https://docs.github.com/rest/reference/actions)
- [activity](https://docs.github.com/rest/reference/activity)
- [api_insights](https://docs.github.com/rest/reference/api-insights)
- [apps](https://docs.github.com/rest/reference/apps)
- [billing](https://docs.github.com/rest/reference/billing)
- [campaigns](https://docs.github.com/rest/reference/campaigns)
- [checks](https://docs.github.com/rest/reference/checks)
- [classroom](https://docs.github.com/rest/reference/classroom)
- [code_scanning](https://docs.github.com/rest/reference/code-scanning)
- [code_security](https://docs.github.com/rest/reference/code-security)
- [codes_of_conduct](https://docs.github.com/rest/reference/codes-of-conduct)
- [codespaces](https://docs.github.com/rest/reference/codespaces)
- [copilot](https://docs.github.com/rest/reference/copilot)
- [credentials](https://docs.github.com/rest/reference/credentials)
- [dependabot](https://docs.github.com/rest/reference/dependabot)
- [dependency_graph](https://docs.github.com/rest/reference/dependency-graph)
- [emojis](https://docs.github.com/rest/reference/emojis)
- [gists](https://docs.github.com/rest/reference/gists)
- [git](https://docs.github.com/rest/reference/git)
- [gitignore](https://docs.github.com/rest/reference/gitignore)
- [hosted_compute](https://docs.github.com/rest/reference/hosted-compute)
- [interactions](https://docs.github.com/rest/reference/interactions)
- [issues](https://docs.github.com/rest/reference/issues)
- [licenses](https://docs.github.com/rest/reference/licenses)
- [markdown](https://docs.github.com/rest/reference/markdown)
- [meta](https://docs.github.com/rest/reference/meta)
- [migrations](https://docs.github.com/rest/reference/migrations)
- [oidc](https://docs.github.com/rest/reference/oidc)
- [orgs](https://docs.github.com/rest/reference/orgs)
- [packages](https://docs.github.com/rest/reference/packages)
- [private_registries](https://docs.github.com/rest/reference/private-registries)
- [projects](https://docs.github.com/rest/reference/projects)
- [projects_classic](https://docs.github.com/rest/reference/projects-classic)
- [pulls](https://docs.github.com/rest/reference/pulls)
- [rate_limit](https://docs.github.com/rest/reference/rate-limit)
- [reactions](https://docs.github.com/rest/reference/reactions)
- [repos](https://docs.github.com/rest/reference/repos)
- [search](https://docs.github.com/rest/reference/search)
- [secret_scanning](https://docs.github.com/rest/reference/secret-scanning)
- [security_advisories](https://docs.github.com/rest/reference/security-advisories)
- [teams](https://docs.github.com/rest/reference/teams)
- [users](https://docs.github.com/rest/reference/users)

In [ ]:
api.codes_of_conduct

- [codes-of-conduct.get_all_codes_of_conduct](https://docs.github.com/rest/codes-of-conduct/codes-of-conduct#get-all-codes-of-conduct)(): *Get all codes of conduct*
- [codes-of-conduct.get_conduct_code](https://docs.github.com/rest/codes-of-conduct/codes-of-conduct#get-a-code-of-conduct)(key): *Get a code of conduct*

### Calling endpoints

The GitHub API's endpoint names generally start with a verb like "get", "list", "delete", "create", etc, followed `_`, then by a noun such as "ref", "webhook", "issue", etc.

Each endpoint has a different signature, which you can see by using <kbd>Shift</kbd>-<kbd>Tab</kbd> in Jupyter, or by just printing the endpoint object (which also shows a link to the GitHub docs):

In [ ]:
print(api.repos.create_webhook)

repos.create_webhook(name: str = None, config: dict = None, events: list = ['push'], active: bool = True)
https://docs.github.com/rest/repos/webhooks#create-a-repository-webhook


Displaying an endpoint object in Jupyter also provides a formatted summary and link to the official GitHub documentation:

In [ ]:
api.repos.create_webhook

[repos.create_webhook](https://docs.github.com/rest/repos/webhooks#create-a-repository-webhook)(name, config, events, active): *Create a repository webhook*

Endpoint objects are called using standard Python method syntax:

In [ ]:
ref = api.git.get_ref('heads/master')
test_eq(ref.object.type, 'commit')

Information about the endpoint are available as attributes:

In [ ]:
api.git.get_ref.path,api.git.get_ref.verb

('/repos/fastai/ghapi-test/git/ref/{ref}', 'get')

You can get a list of all endpoints available in a group, along with a link to documentation for each, by viewing the group:

In [ ]:
api.git

- [git.create_blob](https://docs.github.com/rest/git/blobs#create-a-blob)(content, encoding): *Create a blob*
- [git.get_blob](https://docs.github.com/rest/git/blobs#get-a-blob)(file_sha): *Get a blob*
- [git.create_commit](https://docs.github.com/rest/git/commits#create-a-commit)(message, tree, parents, author, committer, signature): *Create a commit*
- [git.get_commit](https://docs.github.com/rest/git/commits#get-a-commit-object)(commit_sha): *Get a commit object*
- [git.list_matching_refs](https://docs.github.com/rest/git/refs#list-matching-references)(ref): *List matching references*
- [git.get_ref](https://docs.github.com/rest/git/refs#get-a-reference)(ref): *Get a reference*
- [git.create_ref](https://docs.github.com/rest/git/refs#create-a-reference)(ref, sha): *Create a reference*
- [git.update_ref](https://docs.github.com/rest/git/refs#update-a-reference)(ref, sha, force): *Update a reference*
- [git.delete_ref](https://docs.github.com/rest/git/refs#delete-a-reference)(ref): *Delete a reference*
- [git.create_tag](https://docs.github.com/rest/git/tags#create-a-tag-object)(tag, message, object, type, tagger): *Create a tag object*
- [git.get_tag](https://docs.github.com/rest/git/tags#get-a-tag)(tag_sha): *Get a tag*
- [git.create_tree](https://docs.github.com/rest/git/trees#create-a-tree)(tree, base_tree): *Create a tree*
- [git.get_tree](https://docs.github.com/rest/git/trees#get-a-tree)(tree_sha, recursive): *Get a tree*

For "list" endpoints, the noun will be a plural form, e.g.:

In [ ]:
#| hide
for hook in api.repos.list_webhooks():
    api.repos.delete_webhook(hook.id)

In [ ]:
hooks = api.repos.list_webhooks()
test_eq(len(hooks), 0)

You can pass dicts, lists, etc. directly, where they are required for GitHub API endpoints:

In [ ]:
url = 'https://example.com'
cfg = dict(url=url, content_type='json', secret='XXX')
hook = api.repos.create_webhook(config=cfg, events=['ping'])
test_eq(hook.config.url, url)

Let's confirm that our new webhook has been created:

In [ ]:
hooks = api.repos.list_webhooks()
test_eq(len(hooks), 1)
test_eq(hooks[0].events, ['ping'])

Finally, we can delete our new webhook:

In [ ]:
api.repos.delete_webhook(hooks[0].id)

```json
{}
```

### Convenience functions

In [ ]:
#| export
def date2gh(dt:datetime)->str:
    "Convert `dt` (which is assumed to be in UTC time zone) to a format suitable for GitHub API operations"
    return f'{dt.replace(microsecond=0).isoformat()}Z'

The GitHub API assumes that dates will be in a specific string format. `date2gh` converts Python standard `datetime` objects to that format. For instance, to find issues opened in the 'fastcore' repo in the last 4 weeks:

In [ ]:
dt = date2gh(datetime.now(timezone.utc) - timedelta(weeks=4))
issues = GhApi('fastai').issues.list_for_repo(repo='fastcore', since=dt)
len(issues)

0

In [ ]:
#| export
def gh2date(dtstr:str)->datetime:
    "Convert date string `dtstr` received from a GitHub API operation to a UTC `datetime`"
    return datetime.fromisoformat(dtstr.replace('Z', ''))

In [ ]:
# created = issues[0].created_at
# print(created, '->', gh2date(created))

You can set the `debug` attribute to any callable to intercept all requests, for instance to print `Request.summary`. `print_summary` is provided for this purpose. Using this, we can see the preview header that is added for preview functionality, e.g.

In [ ]:
api.debug=print_summary
api.codes_of_conduct.get_all_codes_of_conduct()[0]
api.debug=None

{'data': None,
 'full_url': 'https://api.github.com/codes_of_conduct',
 'headers': {'Accept': 'application/vnd.github.v3+json'},
 'method': 'GET'}


### Preview endpoints

GitHub's preview API functionality requires a special header to be passed to enable it. This is added automatically for you.

## Convenience methods

Some methods in the GitHub API are a bit clunky or unintuitive. In these situations we add convenience methods to `GhApi` to make things simpler. There are also some multi-step processes in the GitHub API that `GhApi` provide convenient wrappers for. The methods currently available are shown below; do not hesitate to [create an issue](https://github.com/fastai/ghapi-test/issues) or pull request if there are other processes that you'd like to see supported better.

In [ ]:
#| export
img_md_pat = re.compile(r'!\[(?P<alt>.*?)\]\((?P<url>[^\s]+)\)')

def _run_subp(cmd): 
    r = subprocess.run(cmd, check=False, capture_output=True, text=True)
    if r.returncode != 0: raise RuntimeError(r.stderr)

@patch
def create_gist(self:GhApi, description, content, filename='gist.txt', public=False, img_paths=None):
    'Create a gist, optionally with images where each md img url will be placed with img upload urls.'
    gist = self.gists.create(description, public=public, files={filename: {"content": content}})
    if not img_paths: return gist
    with tempfile.TemporaryDirectory() as clone_dir:
        token = self.headers['Authorization'].split('token ')[1]
        _run_subp(['git', 'clone', f'https://{token}@gist.github.com/{gist.id}.git', clone_dir])
        clone_dir, img_paths = Path(clone_dir), L(img_paths).map(Path)
        for o in img_paths: shutil.copy2(o, clone_dir/o.name)
        _run_subp(['git', '-C', clone_dir, 'add', '.'])
        _run_subp(['git', '-C', clone_dir, 'commit', '-m', 'Add images'])
        _run_subp(['git', '-C', clone_dir, 'push'])    
    updated_gist = self.gists.get(gist.id)
    img_urls = {o.name: updated_gist.files[o.name].raw_url for o in img_paths}
    content = img_md_pat.sub(lambda m: f"![{m['alt']}]({img_urls.get(m['url'], m['url'])})", content)
    return self.gists.update(gist.id, files={filename:{'content':content}})

In [ ]:
gist = api.create_gist("some description", "some content")
print(gist.html_url)

https://gist.github.com/KeremTurgutlu/c2150e4efd1177439ff35a4633584144


In [ ]:
gist.files['gist.txt'].content

'some content'

In [ ]:
gist = api.create_gist("some description", "some image\n\n![image](puppy.jpg)", 'gist.md', img_paths=['puppy.jpg'])
print(gist.html_url)

https://gist.github.com/KeremTurgutlu/4c60f8b06baf7b57dc225dd86b65028f


In [ ]:
gist.files['gist.md'].content

'some image\n\n![image](https://gist.githubusercontent.com/KeremTurgutlu/4c60f8b06baf7b57dc225dd86b65028f/raw/c7f420c839f58c6ac0c05f1116317645d31d7e80/puppy.jpg)'

Note that if you want to create a gist with multiple files, call the GitHub API directly, e.g.:

```python
api.gists.create("some description", files={"f1.txt": {"content": "my content"}, ...})
```

### Releases

In [ ]:
#| export
@patch
def delete_release(self:GhApi, release):
    "Delete a release and its associated tag"
    self.repos.delete_release(release.id)
    self.git.delete_ref(f'tags/{release.tag_name}')

In [ ]:
#| hide
for rel in api.repos.list_releases(): api.delete_release(rel)

In [ ]:
#| export
@patch
def upload_file(self:GhApi, rel, fn):
    "Upload `fn` to endpoint for release `rel`"
    fn = Path(fn)
    url = rel.upload_url.replace('{?name,label}','')
    mime = mimetypes.guess_type(fn, False)[0] or 'application/octet-stream'
    return self(url, 'POST', headers={'Content-Type':mime}, query = {'name':fn.name}, data=fn.read_bytes())

In [ ]:
#| export
@patch
def create_release(self:GhApi, tag_name, branch='master', name=None, body='',
                   draft=False, prerelease=False, files=None):
    "Wrapper for `GhApi.repos.create_release` which also uploads `files`"
    if name is None: name = 'v'+tag_name
    rel = self.repos.create_release(tag_name, target_commitish=branch, name=name, body=body,
                                   draft=draft, prerelease=prerelease)
    for file in listify(files): self.upload_file(rel, file)
    return rel

Creating a release and attaching files to it is normally a multi-stage process, so `create_release` wraps this up for you. It takes the same arguments as [`repos.create_release`](https://docs.github.com/rest/reference/repos#create-a-release), along with `files`, which can contain a single file name, or a list of file names to upload to your release:

In [ ]:
rel = api.create_release('0.0.1', files=['README.md'])
test_eq(rel.name, 'v0.0.1')

In [ ]:
sleep(0.2)
rels = api.repos.list_releases()
test_eq(len(rels), 1)

We can check that our file has been uploaded; GitHub refers to them as "assets":

In [ ]:
assets = api.repos.list_release_assets(rels[0].id)
test_eq(assets[0].name, 'README.md')

In [ ]:
show_doc(GhApi.delete_release)

---

[source](https://github.com/fastai/ghapi/blob/main/ghapi/core.py#L191){target="_blank" style="float:right; font-size:smaller"}

### GhApi.delete_release

>      GhApi.delete_release (release)

*Delete a release and its associated tag*

### Branches and tags

In [ ]:
#| export
@patch
def list_tags(self:GhApi, prefix:str=''):
    "List all tags, optionally filtered to those starting with `prefix`"
    return self.git.list_matching_refs(f'tags/{prefix}')

With no `prefix`, all tags are listed.

In [ ]:
test_eq(len(api.list_tags()), 1)

Using the full tag name will return just that tag.

In [ ]:
test_eq(len(api.list_tags(rel.tag_name)), 1)

In [ ]:
#| export
@patch
def list_branches(self:GhApi, prefix:str=''):
    "List all branches, optionally filtered to those starting with `prefix`"
    return self.git.list_matching_refs(f'heads/{prefix}')

Branches can be listed in the exactly the same way as tags.

In [ ]:
test_eq(len(api.list_branches('master')), 1)

We can delete our release and confirm that it is removed:

In [ ]:
api.delete_release(rels[0])
test_eq(len(api.repos.list_releases()), 0)

In [ ]:
#| export
# See https://stackoverflow.com/questions/9765453
EMPTY_TREE_SHA = '4b825dc642cb6eb9a060e54bf8d69288fbee4904'

In [ ]:
# #| hide
# #not working
# #| export
# @patch
# def create_branch_empty(self:GhApi, branch):
#     c = self.git.create_commit(f'create {branch}', EMPTY_TREE_SHA)
#     return self.git.create_ref(f'refs/heads/{branch}', c.sha)

In [ ]:
#| export
@patch
def create_branch_empty(self:GhApi, branch):
    t = self.git.create_tree(base_tree=EMPTY_TREE_SHA, tree = [dict(
        path='.dummy', content='ignore me', mode='100644', type='blob')])
    c = self.git.create_commit(f'create {branch}', t.sha)
    return self.git.create_ref(f'refs/heads/{branch}', c.sha)

In [ ]:
ref = api.create_branch_empty("testme")
test_eq(len(api.list_branches('testme')), 1)

In [ ]:
#| export
@patch
def delete_tag(self:GhApi, tag:str):
    "Delete a tag"
    return self.git.delete_ref(f'tags/{tag}')

In [ ]:
#| export
@patch
def delete_branch(self:GhApi, branch:str):
    "Delete a branch"
    return self.git.delete_ref(f'heads/{branch}')

In [ ]:
api.delete_branch('testme')
test_eq(len(api.list_branches('testme')), 0)

In [ ]:
#| export
@patch
def get_branch(self:GhApi, branch=None):
    branch = branch or self.repos.get().default_branch
    return self.list_branches(branch)[0]

### Content (git files)

In [ ]:
#| export
@patch
def list_files(self:GhApi, branch=None):
    ref = self.get_branch(branch)
    res = self.git.get_tree(ref.object.sha).tree
    return {o.path:o for o in res}

In [ ]:
files = api.list_files()
files['README.md']

```json
{ 'mode': '100644',
  'path': 'README.md',
  'sha': 'eaea0f2698e76c75602058bf4e2e9fd7940ac4e3',
  'size': 72,
  'type': 'blob',
  'url': 'https://api.github.com/repos/fastai/ghapi-test/git/blobs/eaea0f2698e76c75602058bf4e2e9fd7940ac4e3'}
```

In [ ]:
#| export
@patch
def get_content(self:GhApi, path):
    res = self.repos.get_content(path)
    return base64.b64decode(res.content)

In [ ]:
readme = api.get_content('README.md').decode()
assert 'ghapi' in readme

In [ ]:
#| export
@patch
def create_or_update_file(self:GhApi, path, message, committer, author, content=None, sha=None, branch=''):
    if not branch: branch = api.repos.get()['default_branch']
    if not isinstance(content,bytes): content = content.encode()
    content = base64.b64encode(content).decode()
    kwargs = {'sha':sha} if sha else {}
    return self.repos.create_or_update_file_contents(path, message, content=content,
        branch=branch, committer=committer or {}, author=author or {}, **kwargs)

In [ ]:
#| export
@patch
def create_file(self:GhApi, path, message, committer, author, content=None, branch=None):
    if not branch: branch = api.repos.get()['default_branch']
    return self.create_or_update_file(path, message, branch=branch, committer=committer, content=content, author=author)

In [ ]:
person = dict(name="Monalisa Octocat", email="octocat@github.com")
res = api.create_file(
    path='foo',
    message="Create foo",
    content="foobar",
    committer=person, author=person
)
test_eq('foobar', api.get_content('foo').decode())

In [ ]:
#| export
@patch
def delete_file(self:GhApi, path, message, committer, author, sha=None, branch=None):
    if not branch: branch = api.repos.get()['default_branch']
    if sha is None: sha = self.list_files()[path].sha
    return self.repos.delete_file(path, message=message, sha=sha,
                                  branch=branch, committer=committer, author=author)

In [ ]:
api.delete_file('foo', 'delete foo', committer=person, author=person)
assert 'foo' not in api.list_files()

In [ ]:
#| export
@patch
def update_contents(self:GhApi, path, message, committer, author, content, sha=None, branch=None):
    if not branch: branch = api.repos.get()['default_branch']
    if sha is None: sha = self.list_files()[path].sha
    return self.create_or_update_file(path, message, committer=committer, author=author, content=content, sha=sha, branch=branch)

In [ ]:
res = api.update_contents(
    path='README.md',
    message="Update README",
    committer=person, author=person,
    content=readme+"foobar"
)
res.content.size

78

In [ ]:
readme = api.get_content('README.md').decode()
assert 'foobar' in readme
api.update_contents('README.md', "Revert README", committer=person, author=person, content=readme[:-6]);

In [ ]:
api = GhApi(token=token)

Let's implement a function to get all valid files of a repo recursively

In [ ]:
@patch
def get_repo_files(self:GhApi, owner, repo, branch="main"):
    "Get all file items of a repo."
    tree = self.git.get_tree(owner=owner, repo=repo, tree_sha=branch, recursive=True)
    res = []
    for item in tree['tree']:
        if item['type'] == 'blob': res.append(item) 
    return L(res)

In [ ]:
owner, repo, branch = "AnswerDotAI", "fastcore", "main"
repo_files = api.get_repo_files(owner,repo); repo_files[:3]

(#3) [{'path': '.devcontainer.json', 'mode': '100644', 'type': 'blob', 'sha': '8bfa0e952eb318c5c74acaa26a0016c12e13418e', 'size': 569, 'url': 'https://api.github.com/repos/AnswerDotAI/fastcore/git/blobs/8bfa0e952eb318c5c74acaa26a0016c12e13418e'},{'path': '.gitattributes', 'mode': '100644', 'type': 'blob', 'sha': '753b249880d57c22306cf155601bff986622b1a0', 'size': 26, 'url': 'https://api.github.com/repos/AnswerDotAI/fastcore/git/blobs/753b249880d57c22306cf155601bff986622b1a0'},{'path': '.github/workflows/docs.yml', 'mode': '100644', 'type': 'blob', 'sha': 'cde13ab17f1a9cbc112928d71ecadee93cf30383', 'size': 296, 'url': 'https://api.github.com/repos/AnswerDotAI/fastcore/git/blobs/cde13ab17f1a9cbc112928d71ecadee93cf30383'}]

It would be useful to add filter options to further filter these files. We can use [fnmatch](https://docs.python.org/3/library/fnmatch.html) to add Unix shell-style wildcard based filtering which is simple yet pretty flexible.

In [ ]:
#| export
def _find_matches(path, pats):
    "Returns matched patterns"
    matches = []
    for p in listify(pats):
        if fnmatch.fnmatch(path,p): matches.append(p)
    return matches

In [ ]:
_find_matches('README.md', ['*.py', '*test_*', '*/test*/*', '*.md', 'README.md'])

['*.md', 'README.md']

In [ ]:
#| export
def _include(path, include, exclude):
    "Prioritize non-star matches, if both include and exclude star expr then pick longer."
    include_matches = ["*"] if include is None else _find_matches(path, include)
    exclude_matches = [] if exclude is None else _find_matches(path, exclude)
    if include_matches and exclude_matches:
        include_star = [m for m in include_matches if "*" in m]
        exclude_star = [m for m in exclude_matches if "*" in m]
        if include_star and exclude_star: return len(include_star) > len(exclude_star)
        if include_star: return False
        if exclude_star: return True    
    if include_matches: return True
    if exclude_matches: return False

Exclude all .md files expect for README.md

In [ ]:
assert _include('README.md', ['README.md'], ['*.md'])
assert not _include('CONTRIBUTING.md', ['README.md'], ['*.md'])

Include all .py files except for tests

In [ ]:
assert not _include('examples/test_fastcore2.py', ['*.py'], ['*test_*', '*/test*/*'])
assert not _include('examples/tests/some_test.py', ['*.py'], ['*test_*', '*/tests/*'])
assert not _include('examples/test/some_test.py', ['*.py'], ['*test_*', '*/test/*'])

In [ ]:
assert _include('cool/module.py', ['*.py'], ['setup.py'])
assert not _include('cool/_modidx', ['*.py'], ['*/_modidx'])
assert not _include('setup.py', ['*.py'], ['setup.py'])

In [ ]:
test_repo_files = ['README.md', 'CONTRIBUTING.md', 'dir/MARKDOWN.md', 'tests/file.py', 
                   'module/file.py', 'module/app/file.py', 'nbs/00.ipynb', 'file2.py',
                   '.gitignore', 'module/.dotfile', '_hidden.py', 'module/_hidden.py']

Here is an example where we filter to include all python files except for the ones under tests directory, include all notebooks, exclude all md files except for README.md, and all files starting with an underscore. 

In [ ]:
inc,exc = ['README.md', '*.py', '*.ipynb'], ['*.md', 'tests/*.py', '_*', '*/_*']
[fn for fn in test_repo_files if _include(fn,inc,exc)]

['README.md',
 'module/file.py',
 'module/app/file.py',
 'nbs/00.ipynb',
 'file2.py']

Let's exclude files starting with `test_` and `setup.py` too.

In [ ]:
exc += ['*test_*.py', '*/*test*.py', 'setup.py']; exc

['*.md', 'tests/*.py', '_*', '*/_*', '*test_*.py', '*/*test*.py', 'setup.py']

The list of files that are kept based on the filtering logic:

In [ ]:
repo_files_filtered = repo_files.filter(lambda o: _include(o.path, inc, exc))
len(repo_files_filtered), list(repo_files_filtered.map(lambda o: o.path))

(39,
 ['README.md',
  'fastcore/all.py',
  'fastcore/ansi.py',
  'fastcore/basics.py',
  'fastcore/dispatch.py',
  'fastcore/docments.py',
  'fastcore/docscrape.py',
  'fastcore/foundation.py',
  'fastcore/imghdr.py',
  'fastcore/imports.py',
  'fastcore/meta.py',
  'fastcore/nb_imports.py',
  'fastcore/net.py',
  'fastcore/parallel.py',
  'fastcore/py2pyi.py',
  'fastcore/script.py',
  'fastcore/shutil.py',
  'fastcore/style.py',
  'fastcore/transform.py',
  'fastcore/utils.py',
  'fastcore/xdg.py',
  'fastcore/xml.py',
  'fastcore/xtras.py',
  'nbs/000_tour.ipynb',
  'nbs/00_test.ipynb',
  'nbs/01_basics.ipynb',
  'nbs/02_foundation.ipynb',
  'nbs/03_xtras.ipynb',
  'nbs/03a_parallel.ipynb',
  'nbs/03b_net.ipynb',
  'nbs/04_docments.ipynb',
  'nbs/05_meta.ipynb',
  'nbs/06_script.ipynb',
  'nbs/07_xdg.ipynb',
  'nbs/08_style.ipynb',
  'nbs/09_xml.ipynb',
  'nbs/10_py2pyi.ipynb',
  'nbs/11_external.ipynb',
  'nbs/index.ipynb'])

Below we can see the files that got filtered out:

In [ ]:
list(repo_files.filter(lambda o: o.path not in repo_files_filtered.attrgot('path')).attrgot('path'))

['.devcontainer.json',
 '.gitattributes',
 '.github/workflows/docs.yml',
 '.github/workflows/main.yml',
 '.gitignore',
 'CHANGELOG.md',
 'CODE_OF_CONDUCT.md',
 'CONTRIBUTING.md',
 'LICENSE',
 'MANIFEST.in',
 'docker-compose.yml',
 'examples/ansi.css',
 'examples/test_fastcore.py',
 'examples/test_fastcore2.py',
 'fastcore/__init__.py',
 'fastcore/_modidx.py',
 'fastcore/_nbdev.py',
 'fastcore/test.py',
 'images/att_00000.png',
 'images/att_00001.png',
 'images/att_00002.png',
 'nbs/.gitattributes',
 'nbs/.gitignore',
 'nbs/.nojekyll',
 'nbs/CNAME',
 'nbs/_parallel_win.ipynb',
 'nbs/_quarto.yml',
 'nbs/fastcore',
 'nbs/files/test.txt.bz2',
 'nbs/images/att_00000.png',
 'nbs/images/att_00005.png',
 'nbs/images/att_00006.png',
 'nbs/images/att_00007.png',
 'nbs/images/mnist3.png',
 'nbs/images/puppy.jpg',
 'nbs/llms-ctx-full.txt',
 'nbs/llms-ctx.txt',
 'nbs/llms.txt',
 'nbs/nbdev.yml',
 'nbs/parallel_test.py',
 'nbs/styles.css',
 'nbs/test_py2pyi.py',
 'nbs/test_py2pyi.pyi',
 'pyproject.t

In [ ]:
from IPython.display import Markdown
item = repo_files_filtered[0]
content = api.repos.get_content(owner, repo, item['path'])
content['content_decoded'] = base64.b64decode(content.content).decode('utf-8')
Markdown(content.content_decoded)

# Welcome to fastcore


<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

Python is a powerful, dynamic language. Rather than bake everything into
the language, it lets the programmer customize it to make it work for
them. `fastcore` uses this flexibility to add to Python features
inspired by other languages we’ve loved, mixins from Ruby, and currying,
binding, and more from Haskell. It also adds some “missing features” and
clean up some rough edges in the Python standard library, such as
simplifying parallel processing, and bringing ideas from NumPy over to
Python’s `list` type.

## Getting started

To install fastcore run: `conda install fastcore -c fastai` (if you use
Anaconda, which we recommend) or `pip install fastcore`. For an
[editable
install](https://stackoverflow.com/questions/35064426/when-would-the-e-editable-option-be-useful-with-pip-install),
clone this repo and run: `pip install -e ".[dev]"`. fastcore is tested
to work on Ubuntu, macOS and Windows (versions tested are those shown
with the `-latest` suffix
[here](https://docs.github.com/en/actions/reference/specifications-for-github-hosted-runners#supported-runners-and-hardware-resources)).

`fastcore` contains many features, including:

- `fastcore.test`: Simple testing functions
- `fastcore.foundation`: Mixins, delegation, composition, and more
- `fastcore.xtras`: Utility functions to help with functional-style
  programming, parallel processing, and more

To get started, we recommend you read through [the fastcore
tour](https://fastcore.fast.ai/tour.html).

## Contributing

After you clone this repository, please run `nbdev_install_hooks` in
your terminal. This sets up git hooks, which clean up the notebooks to
remove the extraneous stuff stored in the notebooks (e.g. which cells
you ran) which causes unnecessary merge conflicts.

To run the tests in parallel, launch `nbdev_test`.

Before submitting a PR, check that the local library and notebooks
match.

- If you made a change to the notebooks in one of the exported cells,
  you can export it to the library with `nbdev_prepare`.
- If you made a change to the library, you can export it back to the
  notebooks with `nbdev_update`.


Let's update `get_repo_files` with the filtering mechanism we've implemented above.

In [ ]:
#| export
@patch
def get_repo_files(self:GhApi, owner, repo, branch="main", inc=None, exc=None):
    "Get all file items of a repo."
    tree = self.git.get_tree(owner=owner, repo=repo, tree_sha=branch, recursive=True)
    res = L()
    for item in tree['tree']:
        if item['type'] == 'blob': res.append(item) 
    return res.filter(lambda o: _include(o.path,inc,exc))

In [ ]:
repo_files = api.get_repo_files(owner, repo, inc=inc, exc=exc); repo_files.attrgot("path")

(#39) ['README.md','fastcore/all.py','fastcore/ansi.py','fastcore/basics.py','fastcore/dispatch.py','fastcore/docments.py','fastcore/docscrape.py','fastcore/foundation.py','fastcore/imghdr.py','fastcore/imports.py','fastcore/meta.py','fastcore/nb_imports.py','fastcore/net.py','fastcore/parallel.py','fastcore/py2pyi.py','fastcore/script.py','fastcore/shutil.py','fastcore/style.py','fastcore/transform.py','fastcore/utils.py'...]

In [ ]:
#| export
@patch
def get_file_content(self:GhApi, path, owner, repo, branch="main"):
    o = self.repos.get_content(owner, repo, path, ref=branch)
    o['content_decoded'] = base64.b64decode(o.content).decode('utf-8')
    return o

In [ ]:
o = api.get_file_content(repo_files[0].path, owner, repo)
_head = "\n".join(o.content_decoded.split("\n")[:5])
print(f"{o.html_url}\n{_head}")

https://github.com/AnswerDotAI/fastcore/blob/main/README.md
# Welcome to fastcore


<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->



In [ ]:
contents = parallel(api.get_file_content, repo_files[:2].attrgot("path"), owner=owner, repo=repo)
for o in contents:
    _head = "\n".join(o.content_decoded.split("\n")[:5])
    print(f"{o.html_url}\n{_head}")

https://github.com/AnswerDotAI/fastcore/blob/main/README.md
# Welcome to fastcore


<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

https://github.com/AnswerDotAI/fastcore/blob/main/fastcore/all.py
from .imports import *
from .foundation import *
from .utils import *
from .parallel import *
from .net import *


In [ ]:
#|export
@patch
@delegates(GhApi.get_repo_files)
def get_repo_contents(self:GhApi, owner, repo, **kwargs):
    repo_files = self.get_repo_files(owner, repo, **kwargs)
    for s in ('inc','exc',): kwargs.pop(s)
    return parallel(self.get_file_content, repo_files.attrgot("path"), owner=owner, repo=repo, **kwargs)

In [ ]:
inc,exc = ['*.md', "*.py"],['*/_*.py', '*test*.py', '*/*test*.py', 'setup.py']

In [ ]:
contents = api.get_repo_contents(owner,repo,branch="main",inc=inc, exc=exc)

In [ ]:
for o in contents:
    _head = "\n".join(o.content_decoded.split("\n")[:5])
    print(f"{o.html_url}\n{_head}")

https://github.com/AnswerDotAI/fastcore/blob/main/CHANGELOG.md
# Release notes

<!-- do not remove -->

## 1.8.1
https://github.com/AnswerDotAI/fastcore/blob/main/CODE_OF_CONDUCT.md
# Contributor Covenant Code of Conduct

## Our Pledge

In the interest of fostering an open and welcoming environment, we as
https://github.com/AnswerDotAI/fastcore/blob/main/CONTRIBUTING.md
# How to contribute

## How to get started

Clone the `fastcore` repository.
https://github.com/AnswerDotAI/fastcore/blob/main/README.md
# Welcome to fastcore


<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

https://github.com/AnswerDotAI/fastcore/blob/main/fastcore/all.py
from .imports import *
from .foundation import *
from .utils import *
from .parallel import *
from .net import *
https://github.com/AnswerDotAI/fastcore/blob/main/fastcore/ansi.py
"Filters for processing ANSI colors."

# Copyright (c) IPython Development Team.
# Modifications by Jeremy Howard.

https://github.com/AnswerDotAI/fastcore/blo

In [ ]:
contents = api.get_repo_contents(owner,"ghapi",branch="main",inc=inc, exc=exc)

In [ ]:
for o in contents:
    _head = "\n".join(o.content_decoded.split("\n")[:5])
    print(f"{o.html_url}\n{_head}")

https://github.com/AnswerDotAI/ghapi/blob/main/.github/scripts/build-tweet.py
import tweetrel
tweetrel.send_tweet()
https://github.com/AnswerDotAI/ghapi/blob/main/CHANGELOG.md
# Release notes

<!-- do not remove -->

## 1.0.6
https://github.com/AnswerDotAI/ghapi/blob/main/CONTRIBUTING.md
# How to contribute

## How to get started

Before anything else, please install the git hooks that run automatic scripts during each commit and merge to strip the notebooks of superfluous metadata (and avoid merge conflicts). After cloning the repository, run the following command inside it:
https://github.com/AnswerDotAI/ghapi/blob/main/README.md
# ghapi


<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

https://github.com/AnswerDotAI/ghapi/blob/main/examples/build.py
#!/usr/bin/env python
from ghapi.build_lib import *
build_funcs()

https://github.com/AnswerDotAI/ghapi/blob/main/ghapi/actions.py
"""Functionality for helping to create GitHub Actions workflows in Python"""

# AUTOGENERATED

### GitHub Pages

In [ ]:
#| export
@patch
def enable_pages(self:GhApi, branch=None, path="/"):
    "Enable or update pages for a repo to point to a `branch` and `path`."
    if path not in ('/docs','/'): raise Exception("path not in ('/docs','/')")
    r = self.repos.get()
    branch = branch or r.default_branch
    source = {"branch": branch, "path": path}
    if r.has_pages: return # self.repos.update_information_about_pages_site(source=source)
    if len(self.list_branches(branch))==0: self.create_branch_empty(branch)
    return self.repos.create_pages_site(source=source)

`branch` is set to the default branch if `None`. `path` must be `/docs` or `/`.

In [ ]:
res = api.enable_pages(branch='new-branch', path='/')

test_eq(res.source.branch, 'new-branch')
test_eq(res.source.path, '/')

api.repos.delete_pages_site()
api.delete_branch('new-branch')

```json
{}
```

## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()